In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import os
import pathlib
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

from tables import DataTrustedIdentifier, InfoImage, create_info_image
from database import connect, create_table_if_not_exists
from sqlalchemy import and_
from sqlalchemy.orm import Session

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Database connection

In [10]:
session: Session
engine, session = connect()
engine.echo=False

2023-03-19 19:59:32,503 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-19 19:59:32,504 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 19:59:32,505 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-19 19:59:32,505 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 19:59:32,506 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-19 19:59:32,506 INFO sqlalchemy.engine.Engine [raw sql] {}


## Create table info_image and fill table

In [ ]:
def fill_table_info_image(color_mode, image_size, result, seq):
    for r in result:
        record_exist = session.query(InfoImage) \
            .filter(and_(InfoImage.color_mode == color_mode,
                         InfoImage.image_size == image_size,
                         InfoImage.path_image == r,
                         InfoImage.seq_id == seq)) \
            .all()
        if len(record_exist) == 0:
            print('inserted new record')
            image_info = create_info_image(color_mode, image_size, r, seq)
            session.add(image_info)


def set_path_image(color_mode, image_size, list_images, query):
    for i, q in enumerate(query):
        barcode = q[0]
        seq = q[1]
        result = sorted([str(image) for image in list_images if barcode in str(image.stem) and os.path.exists(image)])
        print('founded %d files' % len(result))
        if len(result) > 0:
            fill_table_info_image(color_mode, image_size, result, seq)

In [ ]:
create_table_if_not_exists(engine, InfoImage)

for color in ['rgb', 'grayscale']:
    for image_size in ['256', '400', '512']:
        column_name = 'path_%s_%s' % (image_size, color)
        path_fotos = '/home/xandao/Documentos/%s/%s/w_pred_mask' % (color.upper(), image_size)
        list_images = [image for image in pathlib.Path(path_fotos).rglob('*.jpeg') if image.is_file()]
        columns = [DataTrustedIdentifier.barcode, DataTrustedIdentifier.seq]
        query = session.query(*columns).all()
        if len(list_images) > 0 and len(query) >= 0:
            set_path_image(color, image_size, list_images, query)

## Close connection

In [ ]:
session.close()
engine.close()